# __NEW ROADMAP__ (Irrelevant. see next sections for detailed explanation)

https://github.com/openai/spinningup/blob/master/spinup/algos/tf1/trpo/core.py

*   TRPO implemented using diagonal multivariate normal distributions (looking at two sources and some info in OG paper)
*   gradient is calculated as usual, it will be remapped via inverse Hessian
*   objective function contains KL div constraint, which is a Hessian of KL divergence.
*   It can be computed fast and ez with conjugate gradient (CG). But we need to calculate KL div. Unlike using Fisher Information Matrix (FIM)

STEPS

0.  Run the experiment (this is somewhere inside point 1.)
1.  Define MLP Gaussian policy
    *   it takes in list of states and list of actions, which were conducted based on states using old version of policy
    *   MLP produces mean value parameter for each action thats possible in a given state.

        so if state list is of shape (steps, num_observables), MLP will produce parameters of shape (steps, num_actions). only mean. stdev is unique for each action

    *   we sample actions using reparametrization trick: 
        $$a_\pi \sim \mu(states) + \N(\theta)*\sigma$$
        are these logits? im not sure. should be same shape as \mu-> (steps, num_actions)

    *   compute log likelihood of $a_\pi$, sum over actions dim (might be multiplication in log representation)

    *   compute log likelihood for observed actions $a$. This implies that dims of $a$ and $a_\pi$ are the same so no distribution over actions? Not possible, since mu has last dim of num_actions.

    *   retrieve $\mu$ and $\sigma$ of old policy. What to do if num states were different??

    *   compute analytic KL divergence using old and new distribution parameters. 



2.  Compute Surrogate reward
    *   ratio of log likelihoods new/old
    *   times advantage -> objective f-n
3.  Update weights
    *   get MLP params
    *   calculate grad w.r.t. objective f-n
    *   define hessian-vector product function for arbitrary vector
    *   perform CG iterations to get direction and initial step size
    *   backtracking line search

4. update value function (baseline)


## Objective function - surrogate reward $U(\theta)$


(https://www.youtube.com/watch?v=KjWF8VIMGiY)

*change section later* 

We can define a loss which, when optimized, will produce an incremental improvement to a policy with respect to old policy version.

Given we have gathered trajectory using old policy, we can evaluate expected rewards using new policy via _Importance Sampling_.

My explanation can be seen midway throughout [Sutton_P1_Ch05_Monte_Carlo.ipynb notebook (off-policy)](../../RL_Reinforced_Learning/Sutton_P1_Ch05_Monte_Carlo.ipynb)

$$U(\theta) = \mathbb{E}_{\tau \sim \theta_{old}} \bigg[ \frac{P(\tau|\theta)}{P(\tau|\theta_{old})} R(\tau)\bigg]$$
> Using reward for whole trajectory $R(\tau)$ is naive. Usually it is a function of time $R(t)$ -  a cumulative reward sum.

## Gradient $\vec{g}$


Gradient of this expected return is:
$$\vec{g} = \nabla_{\theta}U(\theta) = \mathbb{E}_{\tau \sim \theta_{old}} \bigg[ \frac{\nabla_{\theta} P(\tau|\theta)}{P(\tau|\theta_{old})} R(\tau)\bigg]$$
***
_if old policy and new policy are the same $(\theta = \theta_{old} \ ; \ P(\tau|\theta) = P(\tau|\theta_{old}))$, via_
$$\nabla_{\theta} \ log P(\tau|\theta) = \frac{\nabla_{\theta}P(\tau|\theta)}{P(\tau|\theta)}$$
by expanding probability of trajectory and taking a derivative (system dynamics drops out)
$$\nabla_{\theta} \ log P(\tau|\theta) = \sum_{t=0}^{T} \nabla_{\theta} \ log \ \pi_\theta(a_t|s_t)$$
_we recover 'vanilla' policy gradient ([Simplest_Policy_Gradient](01_Simplest_Policy_Gradient_Implementations.ipynb)):_
$$\nabla_{\theta} J(\pi_{\theta})  = \underset{\tau \sim \pi}{\mathbb{E}}\bigg[ \sum_{t=0}^{T-1} \nabla_{\theta} \ log  \ \pi_\theta(a_t|s_t) \cdot R(\tau) \bigg]$$

_One may say that case $\theta = \theta_{old}$ is a first (linear) term in approximation of $\nabla_{\theta}U(\theta)$. (not sure about this)_
***
If policies are different ($\theta \neq \theta_{old}$), then gradient is
$$\vec{g} = \nabla_{\theta}U(\theta) = \mathbb{E}_{\tau \sim \theta_{old}} \bigg[ \frac{P(\tau|\theta)}{P(\tau|\theta_{old})} \sum_{t=0}^{T-1} \nabla_{\theta} \ log \ \pi_\theta(a_t|s_t) \cdot R(\tau)\bigg]$$
Which looks like importance-sampling version of "Vanilla Gradient"
***
***

## KL divergence constraint


Similar to Natural Policy Gradient (NPG) [02_Natural_Policy_Gradient.ipynb](02_Natural_Policy_Gradient.ipynb), we want to constraint new policy to be similar to old policy. Again, we will do it via KL divergence 
$$D_{KL}(\pi||\pi_{old}) \leq \epsilon$$
_Might better to write it for all possible actions $\pi(\cdot,s)$ and we want to limit maximal divergence encountered in any state_$s$
$$ \max_s D_{KL}(\pi(\cdot,s)|| \pi_{old}(\cdot,s)) \leq \epsilon$$
_But in practice we might take __average__ over all $s$ instead_

In NPG we used quadratic approximation to KL divergence (and we will do it again) in form of Fisher Information Matrix (FIM) ([Fisher_Information.ipynb](../../Statistics/Fisher_Information.ipynb)) which reused 
$$\nabla_{\theta} \ log \ \pi_\theta(a_t|s_t)$$
needed for a gradient.

Alternative approach (same but different) is to express KL divergence in terms of its Hessian $H$ ([KL_Divergence.ipynb](../../Statistics/KL_Divergence.ipynb))
$$\boxed{D_{KL}\bigg(p(x; \theta) ||p(x; \theta + \delta) \bigg) \overset{\text{2nd order}}{\approx} \frac{1}{2}\delta^T \underbrace{H}_{\nabla^2 D_{KL}} \delta = \frac{1}{2}(\theta - \theta_{old})^T H (\theta - \theta_{old}) }$$
But this requires us to actually calculate $D_{KL}$. We will see how later.
***

## Policy update

Because policy parameter update rule is the same (except we use Hessian instead of FIM)
$$\theta_{t+1} = \theta_t + \sqrt{\frac{2\epsilon}{\vec{g}^T {H^{-1}} \vec{g}}} H^{-1}\vec{g}$$
Term $H^{-1}\vec{g}$ can be calculated (or approximated) via Conjugate Gradient (CG) method ([Notes_Method_of_Conjugate_Gradient_Descent.ipynb](../../optimization/Notes_Method_of_Conjugate_Gradient_Descent.ipynb)), which iteratively solves problem 
$$H \vec{x} = \vec{g} \longrightarrow \vec{x}^* = H^{-1} \vec{g}$$
Part of CG iterations is repeated calculation of Hessian-descent direction product $H \vec{v}$.

Because in problem $H \vec{x} = \vec{g}$ involves a Hessian matrix, we can rewrite it as a Jacobian of a gradient
$$H(\cdot)= \underbrace{J}_{\text{Jacobian}}\nabla (\cdot)$$
and we can rewrite Hessian-vector product using only gradients
$$H \vec{v} = J\big(\nabla D_{KL}\big) \vec{v} = J\big(\nabla D_{KL} \cdot \vec{v}\big) $$
This to so called 'Hessian-vector product trick' ([CG_Hessian_vector_trick.ipynb](../../optimization/CG_Hessian_vector_trick.ipynb))

In practice that means that we define a function $$H_v(\vec{v}) = J\big(\nabla D_{KL} \cdot \vec{v}\big)$$
where $\nabla D_{KL}$ is precomputed and outer Jacobian 'remembers' computational graph to make repeated calculations fast.
***

## Objective function (revisited)

Since we want to use "Hessian-vector trick", we dont use FIM, so there is no need to calculate
$$\nabla_{\theta} \ log \ \pi_\theta(a_t|s_t)$$
But we see it in definition of $\nabla_{\theta}U(\theta)$.

Instead of defining $\nabla_{\theta}U(\theta)$ explicitly, we can let auto-differentiation library do the 'heavy' lifting and calculate
$$\nabla_{\theta} U(\theta) = 
\nabla_{\theta}
\left( 
    \mathbb{E}_{\tau \sim \theta_{old}} 
        \left[\frac{P(\tau|\theta)}{P(\tau|\theta_{old})} R(\tau)\right]
\right)$$
***

## Importance sampling ratio

Still its obvious that we have to calculate importance sampling ratio
$\rho = \frac{P(\tau|\theta)}{P(\tau|\theta_{old})}$

We know that
$$P(\tau|\pi) = \rho_0(s_0) \cdot \pi(a_0|s_0) \cdot P(s_1|s_0,a_0)\cdot\pi(a_1|s_1)\cdot P(s_2|s_1,a_1)\cdot \dots$$
$$ = \rho_0(s_0)\cdot \prod_{t=0}^{T-1} \pi(a_t|s_t) \cdot P(s_{t+1}|s_t,a_t)$$
and by taking a ratio, system governed dynamics (and statics) $ P(s_{t+1}|s_t,a_t)$ will cancel out leaving
$$\rho = \frac{P(\tau|\theta)}{P(\tau|\theta_{old})} = \prod_{t=0}^{T-1} \frac{\pi(a_t|s_t)}{\pi_{old}(a_t|s_t)}$$

_It is not clear whether importance sampling ratio multiplies whole reward (advantage function), or does it in a manner where prior history is not taken into account, similarly to rewards-to-go:_
$$L(\tau) = \sum_{t=0}^{T-1} \ log  \ \pi_\theta(a_t|s_t) \cdot \bigg(\sum_{t^\prime = t}^{T-1} R(s_{t^\prime}, a_{t^\prime}, s_{t^\prime +1}) -b(s_t)\bigg)$$
_in this case objective function can be defined as_
$$U(\theta) = \mathbb{E}_{\tau \sim \theta_{old}} 
        \left[\prod_{t=0}^{T-1} \frac{\pi(a_t|s_t)}{\pi_{old}(a_t|s_t)} \cdot 
                \bigg( 
                    \underbrace{R(s_{t^\prime}, a_{t^\prime}, s_{t^\prime +1}) -b(s_t)}_{R(t)}
                \bigg)\right]$$
***
***
***

## Policy parametrization

>Reminder:<br>Simple environments with small number of observation space can be dealt with by tabular approach. When observation space becomes large/continuous, we have to use approximations that generalize. Common case is to use neural networks (NNs), in simplest case Multi Layer Perceptron.

Depending on whether action space is discrete or continuous, we will have a different approach on how we build NN and produce actions.

Our goal is to calculate for a policy:
1. probability (likelihood) of a trajectory $\prod_{t=0}^{T-1} \pi(a_t|s_t)$
2. KL divergence $D_{KL}$
***
***

### Discrete actions (easy)

> Here we will consider that agent can take only one action from list on $N_{act}$ actions at a time. 

> In discrete action case actions are simply integers $[0,1,2,\dots]$, an abstraction for what they mean in environment [North, East, North-East,...].

#### Definition of NN

For a policy implemented via NN we want to feed in a state (a list of states) and for NN to produce list of probabilities for each action in that state (for each state in a list).
$$\vec{\pi }(s)= \{\pi(a_1|s),\dots,\pi(a_{N_{act}}|s)\}^T$$
Setting NN to produce probabilities directly is difficult, because $\vec{\pi(s)}$ has specific requirements
$$|\vec{\pi}(s)|_1 = \sum_a \pi(a|s)= 1; \  0\leq \pi(\cdot|s) \leq 1$$
Common method is to produce 'logits' $e_i$ which are turned into probabilities via 'Softmax':
$$\pi(a_i|s) = \frac{\exp(e_{i})}{\sum_{k=1}^{N_{act}} \exp(e_{k})}$$

> In practice we feed a trajectory of states
> $$\vec{s} = \{s_1,\dots,s_N\}^T$$
> and NN (via Softmax) retrieves a matrix 
> $$P:[N \times N_{actions}] = \{\vec{\pi }(s_1),\dots,\vec{\pi }(s_N)\}^T$$ 
> each prob dist for a state can be sampled.

> Additionally we will define logP for KL divergence
> $$\log P = \{\log \vec{\pi }(s_1),\dots,\log \vec{\pi }(s_N)\}^T= \log\begin{bmatrix} \pi(a_1,s_1) & \pi(a_2,s_1) \\ \pi(a_1,s_2) & \pi(a_2,s_2) \end{bmatrix}$$ 
***

#### Sampling actions

Sampling actions 
$$a_i\sim \pi(\cdot|s_i)$$
can be done via categorical distribution.

>In pytorch we can use <br>`torch.distributions.categorical.Categorical(probs=None, logits=None, validate_args=None)`, </br> which accepts input in either form.

> We sample $P$ for what action policy _thinks is the best_ based on current NN parameters $\theta$
> $$\vec{a}_\pi = \{a_{\pi,1},\dots,a_{\pi,N}\}^T$$
> We also feed list of actions that previous version of NN took at steps $\vec{s}$
> $$\vec{a}_{old} = \{a_{old, 1},\dots,a_{old, N}\}^T$$
> We extract new probabilities from $P$ that correspond to $\vec{a}_\pi$ and $\vec{a}_{old}$
> $$\vec{\pi}= \{\pi(a_{\pi, 1}, s_1), \dots, \pi(a_{\pi, N}, s_N)\}$$
> $$\vec{\pi}_{old} = \{\pi(a_{old, 1}, s_1), \dots, \pi(a_{old, N}, s_N)\}$$
> And for convenience in log-form
> $$ \log \vec{\pi}\ ; \ \log \vec{\pi}_{old}$$

#### Likelihood of a trajectory

<!-- Calculation of likelihood of trajectory $\prod_{t=0}^{T-1} \pi(a_t|s_t)$ is trivial. Except, maybe, for need to extract entry relevant to a given entry. -->
As mentioned earlier, its most likely that importance sampling ratio is not multiplied, but before interleaved ith rewards-to-go
$$U(\theta) = \mathbb{E}_{\tau \sim \theta_{old}} 
        \left[
            \prod_{t=0}^{T-1} \frac{\pi(a_t|s_t)}{\pi_{old}(a_t|s_t)} \cdot 
                R(t)
        \right]$$
> Which is why we are isolating individual term
>$$\rho_t = \frac{\pi(a_t|s_t)}{\pi_{old}(a_t|s_t)}$$
>by
>$$ \vec{\rho} = \exp \left(\log \vec{\pi}-\log \vec{\pi}_{old}\right) = \exp \left\{log \frac{\pi(a_{\pi, 1})}{\pi(a_{old, 1}, s_1)}, \dots \right\} = \left\{\frac{\pi(a_{\pi, 1})}{\pi(a_{old, 1}, s_1)}, \dots \right\}$$
***

#### KL divergence

Mean KL divergence for discrete case.

Instead of selected probabilities for actions, we have to compare all probabilities. 
> We have computed them previously
$$\log P = \{\log \vec{\pi }(s_1),\dots,\log \vec{\pi }(s_N)\}^T= \log\begin{bmatrix} \pi(a_1,s_1) & \pi(a_2,s_1) \\ \pi(a_1,s_2) & \pi(a_2,s_2) \end{bmatrix}$$
We KL sum divergence over actions in each observed state (axis = -1)
$$D_{KL}(\pi_{old}(\cdot|s_i)||\pi(\cdot|s_i)) =\sum_a \pi_{old}(a|s_i) \bigg(\log \pi_{old}(a|s_i) - \log \pi(a|s_i)\bigg) $$
and take a mean over all states
$$\bar D_{KL}(\pi_{old}||\pi) =   \frac{1}{T}\sum_{i=1}^{T} D_{KL}(\pi_{old}(\cdot|s_i)||\pi(\cdot|s_i))$$
>_For some reason in implementation (https://github.com/openai/spinningup/blob/master/spinup/algos/tf1/trpo/core.py) authors compare old to new policy, not the other way around. Although its not symmetric. Id personally follow derivation of KL div approximation. But ill order it their way._


***
***

### Continuous actions (harder)

All implementations for TRPO where action space is continuous parametrize policy as normal distribution $\mathcal{N}_\pi(\mu,\sigma^2)$.
Specifically multivariate diagonal normal distribution.

> Use of classic distributions allows to pre-compute various parameters analytically, which reduces overall computation time.

#### Definition of NN

If our environment receives $N_{act}$ continuous actions, then policy for each state $s$ should produce $N_{act}$ probability distributions, from which individual actions will sampled.
$$\text{Action dist. for state s} = \bigg\{ \mathcal{N}(\mu_{1},\sigma^2_1), \ \dots, \ \mathcal{N}(\mu_{N_{act}},\sigma^2_{N_{act}}) \bigg\}$$
It is the same as defining multivariate gaussian with set of parameters
$$\vec{\mu} = \{ \mu_1, \dots, \mu_{N_{act}} \}$$
$$\vec{\sigma} = \{\sigma_1, \dots, \sigma_{N_{act}} \}$$

> If given a list of $N$ states, we stack vectors of parameters in a larger $N \times N_{acts}$ matrix.

__NOTE:__ It is not necessary to generate individual $\sigma_i$. Each action can have universal parameter.

_Reminder: Of course these parameters are implicitly dependent on weights $\theta$ of NN $\mu_i = \mu_i(\theta)$._
***

#### Sampling actions

Sampling action values for $\mathcal{N}_\pi(\mu,\sigma^2)$ can be done via "reparemetrization trick", used in Variational Autoencoders (VAE) (its used there to enable backprop):
$$\vec{a} \sim \vec{\mu}+ \mathcal{N}_\pi(\vec{0},\vec{1})\cdot\vec{\sigma}$$
***

#### Likelihood of trajectory

For 1D case likelihood that N i.i.d entries $x_i$ came from gaussian distribution, can be computed as follows ([Gaussian_Properties.ipynb](../../Statistics/Gaussian_Properties.ipynb))
$$L(\sigma^2;\mu, \vec{x}) = \prod_{i=1}^N \mathcal{N}_\pi(x_i, \sigma^2; \mu)= \left(\frac{1}{\sqrt{2\sigma^2\pi}}\right)^N \exp \left(-\frac{1}{2\sigma^2}\sum_{i = 1}^N (x_i-\mu)^2\right)$$
For computation stability we can calcualte log-likelihood:
$$l(\sigma^2;\mu, \vec{x}) = \log \left(L(\sigma^2;\mu, \vec{x})\right)=-\frac{1}{2}\left( N\log 2\pi + 2 N \log \sigma + \left(\frac{\sum_{i = 1}^N x_i-\mu}{\sigma}\right)^2\right) $$
> Reminder: $\mu$ ($\sigma$ is unversal) parametrize state. Main distribution argument is action $x_i = a_i$.<br>
> In practice we will compute log-likelihood for individual state-action ($N=1$)
$$\boxed{l(\sigma_i;\mu_i, a_i) = -\frac{1}{2}\left( \log 2\pi + 2 \log \sigma_i + \left(\frac{a_i-\mu_i}{\sigma_i}\right)^2\right)}$$
Log form is needed for KL divergence. We undo log by exponentiation
$$L(\sigma_i;\mu_i, a_i)= \exp \left(l(\sigma_i;\mu_i, a_i)\right)$$

which allows us to compute importance-sampling ratio
$$\rho_t = \sum_{a}\exp \frac{l(\sigma^2;\mu, a)}{l(\sigma^2_{old};\mu_{old}, a)}$$
We have same considerations of $R(t)$ being time dependent, so we want to keep each each term $\rho_t$ of $\vec{\rho}$ separately.

$$U(\theta) = \mathbb{E}_{\tau \sim \theta_{old}} \bigg[ \frac{P(\tau|\theta)}{P(\tau|\theta_{old})} R(\tau,t)\bigg] \rightarrow
\mathbb{E}_{\tau \sim \theta_{old}} \bigg[ \prod_{t=0}^{T-1} \rho_t \cdot R(t)\bigg] = \frac{1}{T_{all}} \prod_{t=0}^{T_{all}} \rho_t \cdot R(t)$$
> as in previous methods we batch many trajectories together. Have to concatenate rewards/advantages accordingly.
***

#### KL divergence (not complete /correct)

Kl divergence can be calculated analytically for two normal multivariate diagonal distributions
([Gaussian_Properties.ipynb](../../Statistics/Gaussian_Properties.ipynb))
$$\boxed{D_{KL}\left(\mathcal{N}_\pi(\sigma^2_1; \mu_1)||\mathcal{N}_\pi(\sigma^2_2; \mu_2)\right) = \frac{1}{2}\left(\frac{(\mu_1 -\mu_2)^2 +\sigma_1^2}{\sigma_2^2} - 1\right) + \log \sigma_2 - \log \sigma_1}$$
We compute sum over actions
$$D_{KL}(\pi(\cdot|s_i)||\pi_{old}(\cdot|s_i)) = \sum_{i\in \vec{a}} \frac{1}{2}\left(\frac{(\vec{\mu_1}_i-\vec{\mu_2}_i)^2 +\vec{\sigma_1}_i^2}{\vec{\sigma_2}_i^2} - 1\right) + \log \vec{\sigma_2 }_i- \log \vec{\sigma_1}_i$$

<style>
img {
  display: block;
  margin-left: auto;
  margin-right: auto;
}
</style>

<img  src="https://spinningup.openai.com/en/latest/_images/math/5808864ea60ebc3702704717d9f4c3773c90540d.svg" 
      width=460 
      style="background-color:white"/>

step 8. Start unpacking from the end:
1. KL divergence of sample

conjugate gradient method explored in

[Notes_Method_of_Conjugate_Gradient_Descent.ipynb](../../optimization/Notes_Method_of_Conjugate_Gradient_Descent.ipynb)







step 9. uses backtracking line search. notes in

[Notes_Line_search.ipynb](../../optimization/Notes_Line_search.ipynb)